In [1]:
# import required packages

import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
## LOAD DATA

In [3]:
# display parent directory and working directory

print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
print(os.getcwd()+':', os.listdir(os.getcwd()));

C:\Users\sanch\OneDrive\Documents\Graduate School\UT-Austin - MSIS\INF 397 - Statistical Analysis and Learning: ['Data', 'Exams', 'Homework', 'Labs', 'Lectures', 'Project', 'SAL_Spring-2018_Syllabus.pdf', 'Textbooks']
C:\Users\sanch\OneDrive\Documents\Graduate School\UT-Austin - MSIS\INF 397 - Statistical Analysis and Learning\Project: ['.ipynb_checkpoints', '180209, Project Timeline.docx', 'arrest_year_counts.xlsx', 'Chicago_Crimes_2001_to_2004.csv', 'Chicago_Crimes_2005_to_2007.csv', 'Chicago_Crimes_2008_to_2011.csv', 'Chicago_Crimes_2012_to_2017.csv', 'crime_type_counts.xlsx', 'Project Proposal PPT.pptx', 'Project Prototype v2.ipynb', 'SAL proposal.docx', 'SAL proposal.pdf', 'well_A.csv', 'year_counts.xlsx']


In [4]:
# import data

df1 = pd.read_csv("Chicago_Crimes_2001_to_2004.csv", index_col = 0)
df2 = pd.read_csv("Chicago_Crimes_2005_to_2007.csv", index_col = 0)
df3 = pd.read_csv("Chicago_Crimes_2008_to_2011.csv", index_col = 0)
df4 = pd.read_csv("Chicago_Crimes_2012_to_2017.csv", index_col = 0)


C:\Users\sanch\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (9,10,11,12,13,14,16,17,18,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\sanch\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
C:\Users\sanch\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# combine data frames

frames = [df1, df2, df3, df4]
full_dataset = pd.concat(frames)

In [6]:
full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7941286 entries, 879 to 6253474
Data columns (total 22 columns):
ID                      float64
Case Number             object
Date                    object
Block                   object
IUCR                    object
Primary Type            object
Description             object
Location Description    object
Arrest                  object
Domestic                object
Beat                    object
District                object
Ward                    object
Community Area          object
FBI Code                object
X Coordinate            object
Y Coordinate            object
Year                    object
Updated On              object
Latitude                object
Longitude               object
Location                object
dtypes: float64(1), object(21)
memory usage: 1.4+ GB


In [7]:
full_dataset.Year.unique()

array([2004, 2003, 2001, 2002, '2004', '2003', '2002', '2001',
       'Y Coordinate', 41.789832136, 2006, 2005, 2007, 2008, 2009, 2011,
       2010, 2016, 2015, 2012, 2014, 2013, 2017], dtype=object)

In [8]:
# fix similar variables

full_dataset['Year'].replace('2001', 2001,inplace=True)
full_dataset['Year'].replace('2002', 2002,inplace=True)
full_dataset['Year'].replace('2003', 2003,inplace=True)
full_dataset['Year'].replace('2004', 2004,inplace=True)

In [9]:
full_dataset.Year.unique()

array([2004, 2003, 2001, 2002, 'Y Coordinate', 41.789832136, 2006, 2005,
       2007, 2008, 2009, 2011, 2010, 2016, 2015, 2012, 2014, 2013, 2017], dtype=object)

In [10]:
# remove strange values

full_dataset2 = full_dataset[full_dataset.Year != 'Y Coordinate']

In [11]:
full_dataset3 = full_dataset2[full_dataset2.Year != 41.789832136]

In [12]:
## DATA EXPLORATION

In [13]:
full_dataset3.sort_values(by=['Year'])
full_dataset3.Year.value_counts()

2008    852053
2006    794685
2009    783900
2010    700692
2007    621849
2001    568517
2002    490879
2003    475913
2005    455811
2004    388205
2011    352066
2012    335670
2013    306703
2014    274527
2016    265462
2015    262995
2017     11357
Name: Year, dtype: int64

In [14]:
full_dataset3.rename(columns={'Primary Type': 'Primary_Type'}, inplace=True)
full_dataset3.head()

C:\Users\sanch\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


ID Case Number                    Date                Block  \
879   4786321.0    HM399414  01/01/2004 12:01:00 AM    082XX S COLES AVE   
2544  4676906.0    HM278933  03/01/2003 12:00:00 AM      004XX W 42ND PL   
2919  4789749.0    HM402220  06/20/2004 11:00:00 AM  025XX N KIMBALL AVE   
2927  4789765.0    HM402058  12/30/2004 08:00:00 PM   045XX W MONTANA ST   
3302  4677901.0    HM275615  05/01/2003 01:00:00 AM   111XX S NORMAL AVE   

      IUCR                Primary_Type                     Description  \
879   0840                       THEFT   FINANCIAL ID THEFT: OVER $300   
2544  2825               OTHER OFFENSE         HARASSMENT BY TELEPHONE   
2919  1752  OFFENSE INVOLVING CHILDREN   AGG CRIM SEX ABUSE FAM MEMBER   
2927  0840                       THEFT   FINANCIAL ID THEFT: OVER $300   
3302  0841                       THEFT  FINANCIAL ID THEFT:$300 &UNDER   

     Location Description Arrest Domestic              ...               Ward  \
879             RESIDENCE  False    False              ...                  7   
2544            RESIDENCE  False     True              ...                 11   
2919            RESIDENCE  False    False              ...                 35   
2927                OTHER  False    False              ...                 31   
3302            RESIDENCE  False    False              ...                 34   

     Community Area FBI Code X Coordinate Y Coordinate  Year  \
879              46       06          NaN          NaN  2004   
2544             61       26  1.17397e+06  1.87676e+06  2003   
2919             22       20          NaN          NaN  2004   
2927             20       06          NaN          NaN  2004   
3302             49       06  1.17495e+06  1.83105e+06  2003   

                  Updated On Latitude Longitude                       Location  
879   08/17/2015 03:03:40 PM      NaN       NaN                            NaN  
2544  04/15/2016 08:55:02 AM  41.8172  -87.6373  (41.817229156, -87.637328162)  
2919  08/17/2015 03:03:40 PM      NaN       NaN                            NaN  
2927  08/17/2015 03:03:40 PM      NaN       NaN                            NaN  
3302  04/15/2016 08:55:02 AM  41.6918  -87.6351  (41.691784636, -87.635115968)  

[5 rows x 22 columns]

In [15]:
full_dataset3['Primary_Type'].replace('NON - CRIMINAL','NON-CRIMINAL',inplace=True)
full_dataset3['Primary_Type'].replace('NON-CRIMINAL (SUBJECT SPECIFIED)','NON-CRIMINAL',inplace=True)

C:\Users\sanch\Anaconda3\lib\site-packages\pandas\core\generic.py:3924: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [16]:
full_dataset3.sort_values(by=['Primary_Type'])
full_dataset3.Primary_Type.value_counts()

THEFT                                1640506
BATTERY                              1442717
CRIMINAL DAMAGE                       922999
NARCOTICS                             885431
OTHER OFFENSE                         491923
ASSAULT                               481661
BURGLARY                              470958
MOTOR VEHICLE THEFT                   370548
ROBBERY                               300453
DECEPTIVE PRACTICE                    280931
CRIMINAL TRESPASS                     229367
PROSTITUTION                           86401
WEAPONS VIOLATION                      77429
PUBLIC PEACE VIOLATION                 58548
OFFENSE INVOLVING CHILDREN             51441
CRIM SEXUAL ASSAULT                    29868
SEX OFFENSE                            28707
GAMBLING                               18806
LIQUOR LAW VIOLATION                   17513
INTERFERENCE WITH PUBLIC OFFICER       15710
ARSON                                  13097
HOMICIDE                                9051
KIDNAPPING

In [17]:
full_dataset3.Arrest.unique()

array([False, True, 'False', 'True'], dtype=object)

In [18]:
full_dataset3['Arrest'].replace('True', True,inplace=True)
full_dataset3['Arrest'].replace('False', False,inplace=True)

C:\Users\sanch\Anaconda3\lib\site-packages\pandas\core\generic.py:3924: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [19]:
full_dataset3.groupby('Year').Arrest.value_counts()

Year  Arrest
2001  False     400628
      True      167889
2002  False     348074
      True      142805
2003  False     334414
      True      141499
2004  False     271524
      True      116681
2005  False     314652
      True      141159
2006  False     550904
      True      243781
2007  False     431718
      True      190131
2008  False     632831
      True      219222
2009  False     562848
      True      221052
2010  False     509189
      True      191503
2011  False     255841
      True       96225
2012  False     245159
      True       90511
2013  False     220484
      True       86219
2014  False     195470
      True       79057
2015  False     193598
      True       69397
2016  False     215076
      True       50386
2017  False       9455
      True        1902
Name: Arrest, dtype: int64

In [20]:
# finalizing dataset

final_dataset = full_dataset3[full_dataset3.Year != 2017]
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7929927 entries, 879 to 6253474
Data columns (total 22 columns):
ID                      float64
Case Number             object
Date                    object
Block                   object
IUCR                    object
Primary_Type            object
Description             object
Location Description    object
Arrest                  bool
Domestic                object
Beat                    object
District                object
Ward                    object
Community Area          object
FBI Code                object
X Coordinate            object
Y Coordinate            object
Year                    object
Updated On              object
Latitude                object
Longitude               object
Location                object
dtypes: bool(1), float64(1), object(20)
memory usage: 1.3+ GB


In [21]:
final_dataset.Year.value_counts()

2008    852053
2006    794685
2009    783900
2010    700692
2007    621849
2001    568517
2002    490879
2003    475913
2005    455811
2004    388205
2011    352066
2012    335670
2013    306703
2014    274527
2016    265462
2015    262995
Name: Year, dtype: int64

In [54]:
final_dataset['Domestic'].replace('True', True,inplace=True)
final_dataset['Domestic'].replace('False', False,inplace=True)

C:\Users\sanch\Anaconda3\lib\site-packages\pandas\core\generic.py:3924: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [55]:
final_dataset.Domestic.unique()

array([False,  True], dtype=bool)

In [22]:
## BUILDING MODELS

In [65]:
refined_data_set = final_dataset[['Year', 'IUCR', 'Primary_Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Block']].copy()

In [66]:
refined_data_set.rename(columns={'Location Description': 'Location_Description'}, inplace=True)

In [64]:
refined_data_set.Location_Description.unique()

array(['RESIDENCE', 'OTHER', 'APARTMENT', 'RESIDENCE PORCH/HALLWAY',
       'GAS STATION', 'COMMERCIAL / BUSINESS OFFICE', 'STREET', 'BANK',
       'SMALL RETAIL STORE', 'DEPARTMENT STORE', 'SIDEWALK',
       'APPLIANCE STORE', 'HOTEL/MOTEL', 'MEDICAL/DENTAL OFFICE',
       'PARKING LOT/GARAGE(NON.RESID.)', 'ALLEY',
       'CHURCH/SYNAGOGUE/PLACE OF WORSHIP', 'DAY CARE CENTER',
       'RESTAURANT', 'COLLEGE/UNIVERSITY GROUNDS',
       'SCHOOL, PUBLIC, BUILDING', 'HOSPITAL BUILDING/GROUNDS',
       'WAREHOUSE', 'FACTORY/MANUFACTURING BUILDING',
       'SCHOOL, PRIVATE, GROUNDS', 'GROCERY FOOD STORE', 'CHA APARTMENT',
       'SCHOOL, PUBLIC, GROUNDS', 'VEHICLE NON-COMMERCIAL',
       'GOVERNMENT BUILDING/PROPERTY', 'AIRPORT/AIRCRAFT',
       'ATM (AUTOMATIC TELLER MACHINE)', 'VACANT LOT/LAND',
       'POLICE FACILITY/VEH PARKING LOT', 'TAVERN/LIQUOR STORE',
       'CHA HALLWAY/STAIRWELL/ELEVATOR', 'RESIDENCE-GARAGE',
       'PARK PROPERTY', 'CHA PARKING LOT/GROUNDS', 'ABANDONED BUILDING'

In [67]:
# Prepare Train Test Split

from sklearn.model_selection import train_test_split
X = refined_data_set[['Year','IUCR', 'Primary_Type', 'Description', 'Location_Description', 'Domestic']].copy()
y = refined_data_set[['Arrest']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [68]:
# Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_entropy = DecisionTreeClassifier(criterion="entropy").fit(X_train,y_train)

dot_data = tree.export_graphviz(tree_entropy, out_file = None, feature_names = list(X), class_names = list(y), filled = True, rounded = True)
graph = graphviz.Source(dot_data)
graph

ValueError: could not convert string to float: 'SMALL RETAIL STORE'